In [3]:
# 5분 주기의 batch

# pykrx to mysql
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()
### 1. 종목 최신화
print(">> 1. 시가총액 5천억 이상 ticker 최신화")
cursor.execute("SELECT ticker FROM stocks_info WHERE cap >= '500000000000'")
tickers = list(map(lambda x: x[0], cursor.fetchall()))

# 1.1. 업데이트 기간 설정
cursor.execute("SELECT DISTINCT date FROM stocks_price ORDER BY date DESC limit 2")
startList = list(map(lambda x: x[0].strftime("%Y%m%d"), cursor.fetchall()))
start = startList[0]
# start = '20210101'
end = str(datetime.now().date()).replace('-', '')
if start == end:
  start = startList[1]
print("1.1. 업데이트 기간 설정 (%s - %s)" %(start, end))
print(start, end)

# 1.2. 기존 데이터 제거
print("1.2. 기존 데이터 제거 (%s - %s)" %(start, end))
cursor.execute("DELETE FROM stocks_price WHERE date >" + start + " AND date <= " + end)
db.commit()
db.close()

# 1.3. 새로운 데이터 업데이트
print("1.3. 새로운 데이터 업데이트 (%s - %s)" %(start, end))
for ticker in tickers:
  try:
    df = stock.get_market_ohlcv_by_date(start, end, ticker)
    time.sleep(0.1)
    df['ticker'] = ticker
    df = df.rename(columns={'티커':'ticker', '시가':'open', '고가':'high', '저가':'low', '종가':'close', '거래량':'volume'})    
    df.index.names = ['date']
    df = df[['ticker', 'open', 'high', 'low', 'close', 'volume']]

    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
    conn = db_connection.connect()
    df.to_sql(name='stocks_price', con=db_connection, if_exists='append', index=True, index_label="date")
    conn.close()
  except Exception as ex:
    print(ex, ticker)
    pass

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()
cursor.execute("DELETE FROM stocks_price WHERE volume = 0")
db.commit()

print(">> 2. 볼린저 밴드 계산")
day = 20

### 2. 볼린저밴드 최신화
# 2.1. 기존 데이터 제거
print("2.1. 기존 데이터 제거 (%s - %s)" %(start, end))
cursor.execute("DELETE FROM boll WHERE date >" + start + " AND date <= " + end)
db.commit()
db.close()

# 2.1. 새로운 데이터 업데이트
print("2.1. 새로운 데이터 업데이트 (%s - %s)" %(start, end))
for ticker in tickers:
  try:
    # Get ticker's price
    cursor.execute("SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' AND date >= '"+ start +"' AND date <= '"+ end + "' ORDER BY date DESC")
    priceDf = pd.DataFrame(cursor.fetchall())
    priceDf = priceDf.rename(columns={0: 'date', 1:'ticker', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'}).drop(['open', 'high', 'low', 'volume'], axis=1)
    # Set Dataframe
    priceDf['period'] = day
    priceDf['low'] = 0
    priceDf['medium'] = 0
    priceDf['high'] = 0
    priceDf['bandWidth'] = 0.0
    priceDf['position'] = 0.0
    # Set bandWidth, Bollinger
    for idx in range(0, len(priceDf)):
      if idx + day > len(priceDf) - 1:
        continue
      copyDf = priceDf[idx:idx+day].loc[:, 'close'].copy()
      avg = copyDf.mean()
      std = copyDf.std()
      
      lo = round(avg - std * 2)
      me = round(avg)
      hi = round(avg + std * 2)
      bw = (hi - lo) / me
      pos = (priceDf.iloc[idx]['close'] - lo) / (hi - lo)
      priceDf.at[idx, 'low'] = lo
      priceDf.at[idx, 'medium'] = me
      priceDf.at[idx, 'high'] = hi
      priceDf.at[idx, 'bandWidth'] = bw
      priceDf.at[idx, 'position'] = pos

    bollDf = priceDf.reset_index().set_index('date').drop(['index'], axis=1)
    
    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
    conn = db_connection.connect()
    bollDf.to_sql(name='boll', con=db_connection, if_exists='append', index=True, index_label="date")        
    conn.close()
  except Exception as ex1:
    print('ex1', ex1)
    pass

### 3. MFI 최신화
# 3.1. 기존 데이터 제거
print("3.1. 기존 데이터 제거 (%s - %s)" %(start, end))
db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()
cursor.execute("DELETE FROM mfi WHERE date >" + start + " AND date <= " + end)
db.commit()
db.close()
day = 10
for ticker in tickers:
  try:
    # Get ticker's price
    cursor.execute("SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' AND date >= '"+ start +"' AND date <= '"+ end + "' ORDER BY date DESC")
    priceDf = pd.DataFrame(cursor.fetchall())
    priceDf = priceDf.rename(columns={0: 'date', 1:'ticker', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'})
    # Set Dataframe
    priceDf['period'] = day
    priceDf['tp'] = 0
    priceDf['mfi'] = 0
    priceDf['mfi_diff'] = 0
    # Set typical price
    for idx in range(0, len(priceDf)):
      row = priceDf.iloc[idx]
      priceDf.at[idx, 'tp'] = round((row['high'] + row['low'] + row['close']) / 3)
    # Set MFI
    for idx in range(0, len(priceDf)):
      if idx + day > len(priceDf) - 1:
        continue
      positiveRMF = 0
      negativeRMF = 0          
      for i in range(idx, idx + day):
        today = priceDf.iloc[i]
        yesterday = priceDf.iloc[i + 1]
        if today['tp'] > yesterday['tp']:
          positiveRMF += today['tp'] * today['volume']
        elif today['tp'] < yesterday['tp']:
          negativeRMF += today['tp'] * today['volume']
      MFI = round(positiveRMF / (positiveRMF + negativeRMF) * 100)
      priceDf.at[idx, 'mfi'] = MFI
      if idx > 0:
        priceDf.at[idx - 1, 'mfi_diff'] = priceDf.iloc[idx - 1]['mfi'] - MFI
    mfiDf = priceDf.reset_index().set_index('date').drop(['index', 'open', 'high', 'low', 'close', 'volume'], axis=1)
    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
    conn = db_connection.connect()
    mfiDf.to_sql(name='mfi', con=db_connection, if_exists='append', index=True, index_label="date")        
    conn.close()
  except Exception as ex1:
    print('ex1', ex1, ticker[0])
    pass

>> 1. 시가총액 5천억 이상 ticker 최신화
1.1. 업데이트 기간 설정 (20210101 - 20211029)
20210101 20211029
1.2. 기존 데이터 제거 (20210101 - 20211029)
1.3. 새로운 데이터 업데이트 (20210101 - 20211029)


In [ ]:
import pandas as pd
import pymysql
import datetime
from sqlalchemy import create_engine

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

n_days = [10, 14]

start = '20190101'
end = '20211025'

try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  for ticker in tickers:
    for day in n_days:
      try:
        # Get ticker's price
        cursor.execute("SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' AND date >= '"+ start +"' AND date <= '"+ end + "' ORDER BY date DESC")
        priceDf = pd.DataFrame(cursor.fetchall())
        priceDf = priceDf.rename(columns={0: 'date', 1:'ticker', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'})
        priceDf['period'] = day
        priceDf['tp'] = 0
        priceDf['mfi'] = 0
        priceDf['mfi_diff'] = 0
        # Set typical price
        for idx in range(0, len(priceDf)):
          row = priceDf.iloc[idx]
          priceDf.at[idx, 'tp'] = round((row['high'] + row['low'] + row['close']) / 3)
        # Set MFI
        for idx in range(0, len(priceDf)):
          if idx + day > len(priceDf) - 1:
            continue
          positiveRMF = 0
          negativeRMF = 0          
          for i in range(idx, idx + day):
            today = priceDf.iloc[i]
            yesterday = priceDf.iloc[i + 1]
            if today['tp'] > yesterday['tp']:
              positiveRMF += today['tp'] * today['volume']
            elif today['tp'] < yesterday['tp']:
              negativeRMF += today['tp'] * today['volume']
          MFI = round(positiveRMF / (positiveRMF + negativeRMF) * 100)
          priceDf.at[idx, 'mfi'] = MFI
          if idx > 0:
            priceDf.at[idx - 1, 'mfi_diff'] = priceDf.iloc[idx - 1]['mfi'] - MFI
        mfiDf = priceDf.reset_index().set_index('date').drop(['index', 'open', 'high', 'low', 'close', 'volume'], axis=1)
        db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
        conn = db_connection.connect()
        mfiDf.to_sql(name='mfi', con=db_connection, if_exists='append', index=True, index_label="date")        
        conn.close()
      except Exception as ex1:
        print('ex1', ex1, ticker[0])
        pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()